In [1]:
import pandas as pd

In [12]:
ea2 = pd.read_csv('bloomberg_data/EA Lithium Carbonate 2 series.csv', sep=';')
ea2 = ea2.drop(columns=['Unnamed: 4', 'Unnamed: 5'])
ea2.head()

,Date XLIX5,XLIX5,Date XLIJ5,XLIJ5
0,17.11.2025,9.78,02.05.2023,37.75
1,14.11.2025,9.73,03.05.2023,36.44
2,13.11.2025,9.71,04.05.2023,36.48
3,12.11.2025,9.68,05.05.2023,36.48
4,11.11.2025,9.63,08.05.2023,36.48


In [22]:
xlix5 = ea2.iloc[:, :2]
xlix5['Date XLIX5'] = pd.to_datetime(xlix5['Date XLIX5'], format='%d.%m.%Y')
xlix5.sort_values('Date XLIX5', inplace=True)
xlix5.reset_index(drop=True, inplace=True)
xlix5.rename(columns={'Date XLIX5': 'Date'}, inplace=True)
xlix5.head()

,Date,XLIX5
0,2023-12-01,17.18
1,2023-12-04,17.30
2,2023-12-05,16.00
3,2023-12-06,16.00
4,2023-12-07,16.95


In [23]:
xlij5 = ea2.iloc[:, 2:4]
xlij5['Date XLIJ5'] = pd.to_datetime(xlij5['Date XLIJ5'], format='%d.%m.%Y')
xlij5.sort_values('Date XLIJ5', inplace=True)
xlij5.reset_index(drop=True, inplace=True)
xlij5.rename(columns={'Date XLIJ5': 'Date'}, inplace=True)
xlij5.head()

,Date,XLIJ5
0,2023-05-02,37.75
1,2023-05-03,36.44
2,2023-05-04,36.48
3,2023-05-05,36.48
4,2023-05-08,36.48


In [43]:
import pandas as pd

xlix5['Date'] = pd.to_datetime(xlix5['Date'])
xlij5['Date'] = pd.to_datetime(xlij5['Date'])

# --- STEP 1: FIND TRANSITION DATE FROM OVERLAP ONLY ---
overlap = pd.merge(
    xlix5[['Date', 'XLIX5']],
    xlij5[['Date', 'XLIJ5']],
    on='Date',
    how='inner'
)

overlap['diff'] = (overlap['XLIX5'] - overlap['XLIJ5']).abs()
overlap['diff_ma5'] = overlap['diff'].rolling(5).mean()

best_date = overlap.loc[overlap['diff_ma5'].idxmin(), 'Date']

# --- STEP 2: OUTER MERGE TO KEEP ALL DATES ---
merged = pd.merge(
    xlix5[['Date', 'XLIX5']],
    xlij5[['Date', 'XLIJ5']],
    on='Date',
    how='outer'
).sort_values('Date')

# --- STEP 3: BUILD FINAL COMBINED SERIES ---
# before transition → XLIX5  
# from transition onward → XLIJ5

def pick_value(row):
    if row['Date'] < best_date:
        return row['XLIJ5']
    else:
        return row['XLIX5']

merged['Combined'] = merged.apply(pick_value, axis=1)

# --- SAVE TO CSV ---
merged[['Date', 'Combined']].to_csv('east_asia_combined.csv', index=False)

print("Transition performed.")
print("Transition date:", best_date)


Transition performed.
Transition date: 2024-04-22 00:00:00


In [ ]:
eabg2 = pd.read_csv('bloomberg_data/EA Battery Grade.csv', sep=';')
eabg2 = eabg2.drop(columns=['Unnamed: 4', 'Unnamed: 5'])
eabg2.head()

,Date PLIH5,PLIH5,Date PLIX5,PLIX5
0,4/3/23,43.70,11/18/25,9.78
1,4/4/23,42.94,11/17/25,9.73
2,4/5/23,42.92,11/14/25,9.71
3,4/6/23,40.36,11/13/25,9.68
4,4/10/23,38.67,11/12/25,9.63


In [32]:
plih5 = eabg2.iloc[:, :2]
plih5['Date PLIH5'] = pd.to_datetime(plih5['Date PLIH5'], format='mixed')
plih5.sort_values('Date PLIH5', inplace=True)
plih5.reset_index(drop=True, inplace=True)
plih5.rename(columns={'Date PLIH5': 'Date'}, inplace=True)
plih5.head()

,Date,PLIH5
0,2023-04-03,43.70
1,2023-04-04,42.94
2,2023-04-05,42.92
3,2023-04-06,40.36
4,2023-04-10,38.67


In [33]:
plix5 = eabg2.iloc[:, 2:4]
plix5['Date PLIX5'] = pd.to_datetime(plix5['Date PLIX5'], format='mixed')
plix5.sort_values('Date PLIX5', inplace=True)
plix5.reset_index(drop=True, inplace=True)
plix5.rename(columns={'Date PLIX5': 'Date'}, inplace=True)
plix5.head()

,Date,PLIX5
0,2023-12-01,19.45
1,2023-12-04,17.18
2,2023-12-05,17.30
3,2023-12-06,16.00
4,2023-12-07,16.00


In [47]:
import pandas as pd

plih5['Date'] = pd.to_datetime(plih5['Date'])
plix5['Date'] = pd.to_datetime(plix5['Date'])

# --- STEP 1: INNER JOIN FOR TRANSITION DATE ---
overlap_bg = pd.merge(
    plih5[['Date', 'PLIH5']],
    plix5[['Date', 'PLIX5']],
    on='Date',
    how='inner'
)

overlap_bg['diff'] = (overlap_bg['PLIH5'] - overlap_bg['PLIX5']).abs()
overlap_bg['diff_ma5'] = overlap_bg['diff'].rolling(5).mean()

best_date_bg = overlap_bg.loc[overlap_bg['diff_ma5'].idxmin(), 'Date']

# save to csv to verify
overlap_bg.to_csv('battery_grade_overlap.csv', index=False)

# --- STEP 2: OUTER JOIN TO KEEP ALL DATES ---
merged_bg = pd.merge(
    plih5[['Date', 'PLIH5']],
    plix5[['Date', 'PLIX5']],
    on='Date',
    how='outer'
).sort_values('Date')

# --- STEP 3: BUILD FINAL COMBINED SERIES ---
merged_bg['Combined'] = merged_bg.apply(
    lambda row: row['PLIH5'] if row['Date'] < best_date_bg else row['PLIX5'],
    axis=1
)

# --- SAVE ---
merged_bg[['Date', 'Combined']].to_csv('east_asia_battery_grade_combined.csv', index=False)

print("Battery Grade Transition performed.")
print("Best transition date:", best_date_bg)

Battery Grade Transition performed.
Best transition date: 2023-12-29 00:00:00


In [41]:
merged_bg

,Date,PLIH5,PLIX5,diff,diff_ma5,Combined
0,2023-12-01,17.93,19.45,1.52,NaN,19.45
1,2023-12-04,16.47,17.18,0.71,NaN,17.18
2,2023-12-05,16.56,17.30,0.74,NaN,17.30
3,2023-12-06,15.77,16.00,0.23,NaN,16.00
4,2023-12-07,15.28,16.00,0.72,0.784,16.00
...,...,...,...,...,...,...
337,2025-03-25,9.62,11.01,1.39,1.324,9.62
338,2025-03-26,9.62,10.98,1.36,1.320,9.62
339,2025-03-27,9.62,10.98,1.36,1.322,9.62
340,2025-03-28,9.46,10.85,1.39,1.348,9.46
